# 1. 单局 PK 可视化区域

In [1]:
from gomoku.player import ZeroMCTSPlayer, WrongZeroMCTSPlayer, IneffectiveZeroMCTSPlayer, play_one_game
from gomoku.gomoku_env import GomokuEnvSimple
from gomoku.policy import ZeroPolicy

/home/zhangpeng.pada/GomokuZero/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-07 22:03:53,777	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
from gomoku.player import IneffectiveZeroMCTSPlayer
import torch

policy = ZeroPolicy(board_size=9)
policy.load_state_dict(torch.load('models/gomoku_zero_effective_zero_with_data_improving/policy_step_12500.pth'))
player1 = ZeroMCTSPlayer(policy, itermax=200, device='cpu', eager=True)
player2 = ZeroMCTSPlayer(policy, itermax=200, device='cpu', eager=True)

game = GomokuEnvSimple()

info = play_one_game(player1, player2, game, render=True)

   0  1  2  3  4  5  6  7  8
 0 .  .  .  .  .  .  .  .  . 
 1 .  .  .  .  .  .  .  .  . 
 2 .  .  .  .  .  .  .  .  . 
 3 .  .  .  .  .  .  .  .  . 
 4 .  .  .  .  X  .  .  .  . 
 5 .  .  .  .  .  .  .  .  . 
 6 .  .  .  .  .  .  .  .  . 
 7 .  .  .  .  .  .  .  .  . 
 8 .  .  .  .  .  .  .  .  . 

   0  1  2  3  4  5  6  7  8
 0 .  .  .  .  .  .  .  .  . 
 1 .  .  .  .  .  .  .  .  . 
 2 .  .  .  .  .  .  .  .  . 
 3 .  .  .  .  O  .  .  .  . 
 4 .  .  .  .  X  .  .  .  . 
 5 .  .  .  .  .  .  .  .  . 
 6 .  .  .  .  .  .  .  .  . 
 7 .  .  .  .  .  .  .  .  . 
 8 .  .  .  .  .  .  .  .  . 

   0  1  2  3  4  5  6  7  8
 0 .  .  .  .  .  .  .  .  . 
 1 .  .  .  .  .  .  .  .  . 
 2 .  .  .  .  .  .  .  .  . 
 3 .  .  .  .  O  .  .  .  . 
 4 .  .  .  .  X  .  .  .  . 
 5 .  .  .  .  .  X  .  .  . 
 6 .  .  .  .  .  .  .  .  . 
 7 .  .  .  .  .  .  .  .  . 
 8 .  .  .  .  .  .  .  .  . 

   0  1  2  3  4  5  6  7  8
 0 .  .  .  .  .  .  .  .  . 
 1 .  .  .  .  .  .  .  .  . 
 2 .  .  . 

In [ ]:
env = GomokuEnvSimple()
env.board[0, 1:3] = 1
env.board[0, 7] = 1
env.board[1, 1:4] = 2
env.last_action = 10

r = player1.play(env)

# policy(env._get_observation())

# 2. ARENA 

In [ ]:
from gomoku.player import arena_parallel
from gomoku.player import ZeroMCTSPlayer, WrongZeroMCTSPlayer, IneffectiveZeroMCTSPlayer, play_one_game
from gomoku.gomoku_env import GomokuEnvSimple
from gomoku.policy import ZeroPolicy

In [ ]:
r = arena_parallel(
    player1_main=ZeroMCTSPlayer(ZeroPolicy(board_size=9), itermax=50, device='cpu', eager=False),
    player2_main=WrongZeroMCTSPlayer(ZeroPolicy(board_size=9), itermax=50, device='cpu', eager=False),
    games=100,
)

In [ ]:
r = arena_parallel(
    player1_main=ZeroMCTSPlayer(ZeroPolicy(board_size=9), itermax=100, device='cpu', eager=False),
    player2_main=IneffectiveZeroMCTSPlayer(ZeroPolicy(board_size=9), itermax=100, device='cpu', eager=False),
    games=50,
)


# 3. 评价棋局

In [ ]:
from gomoku.player import ZeroMCTSPlayer, WrongZeroMCTSPlayer, IneffectiveZeroMCTSPlayer, play_one_game, self_play
from gomoku.gomoku_env import GomokuEnvSimple
from gomoku.policy import ZeroPolicy
import torch

In [ ]:
policy = ZeroPolicy(board_size=9)
policy.load_state_dict(torch.load('models/gomoku_zero_effective_zero_with_data_improving/policy_step_10500.pth'))

In [ ]:
infos = self_play(policy, 'cpu', 200)

In [ ]:
def render(states, index):
    import numpy as np
    states_tensor = torch.from_numpy(np.array(infos['states'], dtype=np.float32))
    probs, winrate = policy(states_tensor)

    symbols = {0: '.', 1: 'X', 2: 'O'}
    board_str = "  " + " ".join([f"{i:2d}" for i in range(9)]) + "\n"
    
    for i in range(9):
        cells = []
        for j in range(9):
            if states[index][ 0, i, j] ==  1:  
                cell = 1
            elif states[index][ 1, i, j] == 1:
                cell = 2
            else:
                cell = 0
            cells.append(cell)
        board_str += f"{i:2d} " + " ".join([f"{symbols[cell]} " for cell in cells]) + "\n"
    print(board_str)
    print(winrate[index])
    values, indices = torch.topk(torch.softmax(probs, dim=-1)[index], 10)

    def action_2_index(action):
        return action // 9, action % 9

    for i  in range(len(indices)):
        a, b = action_2_index(indices[i].item())
        print(f"[{a:2d},{b:2d}]:{values[i].item():.4f}")


In [ ]:
render(infos['states'], 6)

In [156]:
render(info[1]['states'], -2)

   0  1  2  3  4  5  6  7  8
 0 .  .  .  .  .  .  .  .  . 
 1 .  O  .  .  .  .  .  .  . 
 2 .  .  X  .  .  .  .  O  . 
 3 .  .  .  X  .  .  O  .  . 
 4 .  .  .  .  X  O  .  .  . 
 5 .  .  .  X  O  X  .  .  . 
 6 .  .  .  X  .  .  O  .  . 
 7 .  .  .  .  .  .  .  .  . 
 8 .  .  .  .  .  .  .  .  . 

tensor([-0.8022], grad_fn=<SelectBackward0>)
[ 2, 4]:0.0753
[ 6, 4]:0.0741
[ 7, 7]:0.0535
[ 3, 8]:0.0437
[ 1, 1]:0.0413
[ 2, 8]:0.0327
[ 7, 0]:0.0327
[ 5, 0]:0.0295
[ 1, 8]:0.0295
[ 5, 6]:0.0280


In [157]:
def action_2_index(action):
    return action // 9, action % 9

probs = torch.tensor(info[1]['probs'])
values, indices = probs[-2].topk(10)

for i  in range(len(indices)):
    a, b = action_2_index(indices[i].item())
    print(f"[{a:2d},{b:2d}]:{values[i].item():.4f}")


[ 4, 3]:0.2713
[ 7, 3]:0.2146
[ 1, 5]:0.0810
[ 2, 3]:0.0486
[ 6, 2]:0.0405
[ 6, 5]:0.0243
[ 2, 6]:0.0202
[ 1, 8]:0.0162
[ 7, 0]:0.0121
[ 0, 0]:0.0121
